In [372]:
#Dataframe manipulation library
import pandas as pd
#Math functions, we'll only need the sqrt function so let's import only that
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [373]:
movies_df = pd.read_csv("movies.csv")
ratings_df = pd.read_csv("ratings.csv")

In [374]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [375]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


In [376]:
# Make a different column for years in movies_df

#extracting year with parenthesis
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)

#remove parenthesis from year column
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)', expand=False)

#remove year from title
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')

movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [377]:
# Remove genre column
movies_df = movies_df.drop('genres', 1)
movies_df.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


In [378]:
# Remove timestamp column from ratings_df
ratings_df = ratings_df.drop('timestamp', 1)
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


In [379]:
# User input to select data from
userInput = [
            {'title':'Breakfast Club, The', 'rating':5, 'movieId':1968},
            {'title':'Toy Story', 'rating':3.5, 'movieId':1},
            {'title':'Jumanji', 'rating':2, 'movieId':2},
            {'title':"Pulp Fiction", 'rating':5, 'movieId':296},
            {'title':'Akira', 'rating':4.5, 'movieId':1274}
         ]

In [380]:
# Converting dictionary to DataFrame
userInput_df = pd.DataFrame(userInput)
userInput_df.head(20)

,title,rating,movieId
0,"Breakfast Club, The",5.0,1968
1,Toy Story,3.5,1
2,Jumanji,2.0,2
3,Pulp Fiction,5.0,296
4,Akira,4.5,1274


In [381]:
# creating another DataFrame consisting of users that have movies in common with the input user
userSubset_df = ratings_df[ratings_df['movieId'].isin(userInput_df['movieId'].tolist())]

In [382]:
userSubset_df.head(20)

,userId,movieId,rating
19,4,296,4.0
441,12,1968,3.0
479,13,2,2.0
531,13,1274,5.0
681,14,296,2.0
749,15,1,4.0
776,15,296,3.0
911,15,1968,3.0
1247,17,1,5.0
1248,17,2,3.0


In [383]:
# create several subframes where they all have a common userId in each subframe
userSubsetGroup = userSubset_df.groupby(['userId'])
userSubsetGroup.get_group(75)

,userId,movieId,rating
7507,75,1,5.0
7508,75,2,3.5
7540,75,296,5.0
7633,75,1274,4.5
7673,75,1968,5.0


In [384]:
# we can now sequence the subframes in such a way that the subframes with highest count of elements occur first
userSubsetGroup = sorted(userSubsetGroup, key= lambda x: len(x[1]), reverse = True)
userSubsetGroup[:10]

[(75,
        userId  movieId  rating
  7507      75        1     5.0
  7508      75        2     3.5
  7540      75      296     5.0
  7633      75     1274     4.5
  7673      75     1968     5.0),
 (106,
        userId  movieId  rating
  9083     106        1     2.5
  9084     106        2     3.0
  9115     106      296     3.5
  9198     106     1274     3.0
  9238     106     1968     3.5),
 (686,
         userId  movieId  rating
  61336     686        1     4.0
  61337     686        2     3.0
  61377     686      296     4.0
  61478     686     1274     4.0
  61569     686     1968     5.0),
 (815,
         userId  movieId  rating
  73747     815        1     4.5
  73748     815        2     3.0
  73922     815      296     5.0
  74362     815     1274     3.0
  74678     815     1968     4.5),
 (1040,
         userId  movieId  rating
  96689    1040        1     3.0
  96690    1040        2     1.5
  96733    1040      296     3.5
  96859    1040     1274     3.0
  96922    1

In [385]:
# to reduce complexity in running time, let us limit the userId to top 100 users
userSubsetGroup = userSubsetGroup[0:100]

In [386]:
# storing userId and similarityIndex of each user in userSubsetGroup in a dictionary
# similarityIndex domain [-1,1]

similarityIndexDict = {}

for name, group in userSubsetGroup:
    #first we need to make sure that the movieId in userInput and each subframe of userSubsetGroup is sorted
    group = group.sort_values('movieId')
    inputMovies = userInput_df.sort_values('movieId')
    
    #next we need to get (nRatings) the number of ratings each subframe and the userInput_df have in common
    nRatings = len(group)
    #since we had just chosen the top 100 subframes, it is very likely that nRatings for all is same(5) but we can't assume it
    
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    
    #Ratings from userInput
    tempRatingList = temp_df['rating'].tolist()
    
    #Each ratings frame from subframe 
    tempGroupList = group['rating'].tolist()
    
    #We can calculate the similarityIndex (Pearson correlation)
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        similarityIndexDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        similarityIndexDict[name] = 0
    
    
#print(similarityIndexDict)
#temp_df.head(50)
#tempRatingList
#tempGroupList

In [387]:
similarityIndexDict.items()

dict_items([(75, 0.8272781516947562), (106, 0.5860090386731182), (686, 0.8320502943378437), (815, 0.5765566601970551), (1040, 0.9434563530497265), (1130, 0.2891574659831201), (1502, 0.8770580193070299), (1599, 0.4385290096535153), (1625, 0.716114874039432), (1950, 0.179028718509858), (2065, 0.4385290096535153), (2128, 0.5860090386731196), (2432, 0.1386750490563073), (2791, 0.8770580193070299), (2839, 0.8204126541423674), (2948, -0.11720180773462392), (3025, 0.45124262819713973), (3040, 0.89514359254929), (3186, 0.6784622064861935), (3271, 0.26989594817970664), (3429, 0.0), (3734, -0.15041420939904673), (4099, 0.05860090386731196), (4208, 0.29417420270727607), (4282, -0.4385290096535115), (4292, 0.6564386345361464), (4415, -0.11183835382312353), (4586, -0.9024852563942795), (4725, -0.08006407690254357), (4818, 0.4885967564883424), (5104, 0.7674257668936507), (5165, -0.4385290096535153), (5547, 0.17200522903844556), (6082, -0.04728779924109591), (6207, 0.9615384615384616), (6366, 0.65779

In [388]:
# creating a similarity_df from the similarityIndexDict with keys being userId and values being similarityIndex
similarity_df = pd.DataFrame(similarityIndexDict.keys())
similarity_df.columns = ['userId']
similarity_df['similarityIndex'] = similarityIndexDict.values()

In [389]:
# only 100
similarity_df.head(200)

,userId,similarityIndex
0,75,0.827278
1,106,0.586009
2,686,0.832050
3,815,0.576557
4,1040,0.943456
...,...,...
95,17854,0.537086
96,17897,0.877058
97,17944,0.271385
98,18301,0.298381


In [390]:
# making it in descending order of similarityIndex
similarity_df = similarity_df.sort_values(by = 'similarityIndex', ascending = False)[0:50]

In [391]:
similarity_df.head(50)

,userId,similarityIndex
64,12325,0.961678
34,6207,0.961538
55,10707,0.961538
67,13053,0.960769
4,1040,0.943456
59,11769,0.937614
62,12120,0.929294
80,15157,0.903584
70,13366,0.895144
17,3040,0.895144


In [392]:
# merging movieId and rating provided by each userId
similarity_df = similarity_df.merge(ratings_df, left_on = 'userId', right_on = 'userId', how = 'inner')

In [393]:
similarity_df.head(2000)

,userId,similarityIndex,movieId,rating
0,12325,0.961678,1,3.5
1,12325,0.961678,2,1.5
2,12325,0.961678,3,3.0
3,12325,0.961678,5,0.5
4,12325,0.961678,6,2.5
...,...,...,...,...
1995,13053,0.960769,293,4.0
1996,13053,0.960769,296,4.5
1997,13053,0.960769,306,4.5
1998,13053,0.960769,307,4.5


In [394]:
# making another column weightedRating = rating*similarityIndex
similarity_df['weightedRating'] = similarity_df['similarityIndex'] * similarity_df['rating']

In [395]:
similarity_df.head(2000)

,userId,similarityIndex,movieId,rating,weightedRating
0,12325,0.961678,1,3.5,3.365874
1,12325,0.961678,2,1.5,1.442517
2,12325,0.961678,3,3.0,2.885035
3,12325,0.961678,5,0.5,0.480839
4,12325,0.961678,6,2.5,2.404196
...,...,...,...,...,...
1995,13053,0.960769,293,4.0,3.843076
1996,13053,0.960769,296,4.5,4.323460
1997,13053,0.960769,306,4.5,4.323460
1998,13053,0.960769,307,4.5,4.323460


In [396]:
# calculating sum of similarityIndex and weightedRating and creating columns for it
similarity_df = similarity_df.groupby('movieId').sum()[['similarityIndex','weightedRating']]
similarity_df.columns = ['sum_similarityIndex','sum_weightedRating']

In [397]:
similarity_df.head(2000)

,sum_similarityIndex,sum_weightedRating
movieId,,
1,38.376281,140.800834
2,38.376281,96.656745
3,10.253981,27.254477
4,0.929294,2.787882
5,11.723262,27.151751
...,...,...
2925,1.479076,6.518324
2926,4.618765,15.629836
2927,1.904142,8.804596


In [398]:
# creating 'weighted average recommendation score' column
similarity_df['weighted average recommendation score'] = similarity_df['sum_weightedRating']/similarity_df['sum_similarityIndex']
similarity_df = similarity_df.drop(['sum_similarityIndex', 'sum_weightedRating'], 1)

In [399]:
similarity_df.head(2000)

,weighted average recommendation score
movieId,
1,3.668955
2,2.518658
3,2.657941
4,3.000000
5,2.316058
...,...
2925,4.407023
2926,3.383986
2927,4.623917


In [400]:
# merging similarity_df and movies_df
movies_df = movies_df.merge(similarity_df, left_on='movieId', right_on='movieId', how='inner')

In [401]:
movies_df.head(200)

,movieId,title,year,weighted average recommendation score
0,1,Toy Story,1995,3.668955
1,2,Jumanji,1995,2.518658
2,3,Grumpier Old Men,1995,2.657941
3,4,Waiting to Exhale,1995,3.000000
4,5,Father of the Bride Part II,1995,2.316058
...,...,...,...,...
195,251,"Hunted, The",1995,2.633109
196,252,I.Q.,1994,2.909530
197,253,Interview with the Vampire: The Vampire Chroni...,1994,3.815204
198,254,Jefferson in Paris,1995,2.000000


In [402]:
# sorting 'weighted average recommendation score' in decending order to get the movies with highest average recommendation score
movies_df = movies_df.sort_values(by='weighted average recommendation score', ascending=False)

In [403]:
movies_df.head(60)

,movieId,title,year,weighted average recommendation score
3364,5073,"Son's Room, The (Stanza del figlio, La)",2001,5.0
2241,3329,"Year My Voice Broke, The",1987,5.0
1543,2284,Bandit Queen,1994,5.0
4962,26801,Dragon Inn (Sun lung moon hak chan),1992,5.0
4122,6776,Lagaan: Once Upon a Time in India,2001,5.0
4076,6672,War Photographer,2001,5.0
2505,3759,Fun and Fancy Free,1947,5.0
2514,3769,Thunderbolt and Lightfoot,1974,5.0
2519,3775,Make Mine Music,1946,5.0
7087,90531,Shame,2011,5.0


In [ ]:
# Thus the weighted average score with 5 gives the movies with top user-user collaborative recommendation engine
# These are the top movies which Netflix abreviates as "People who watched this also liked"......